In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import astropy
import matplotlib.pyplot as plt

## Extracción datos simbad

In [2]:
r= requests.get("http://simbad.cds.unistra.fr/simbad/sim-id?Ident=NAME+ASSOC+II+SCO&NbIdent=query_hlinks&Coord=16+15-24.2&children=1121&submit=children&hlinksdisplay=h_all")

In [3]:
soup= bs(r.content)

In [4]:
table= soup.find_all("table", attrs={"id":"datatable"})

df= pd.read_html(str(table))[0]

In [5]:
df.head()

,N,Identifier,dist(asec),Hierarchy probability of membership prob.member,Reference of the hierarchical link link ref,Origin for this hierarchical link : Biblio/Simbad linkcomment,Otype,ICRS (J2000)RA,ICRS (J2000)DEC,Mag U,Mag B,Mag V,Mag R,Mag I,Sp type,#ref 1850 - 2022,#notes
0,1,CD-43 6891,226775.71,100,2016MNRAS.461..794P,B,*,11 14 34.4317850256,-44 18 24.129693828,NaN,10.760,9.730,NaN,NaN,K2IV,2,0
1,2,HD 99627,220604.23,72,2011MNRAS.416.3108R,B,*,11 27 29.3802896784,-39 52 35.444378964,NaN,7.380,7.320,NaN,NaN,A0V,17,0
2,3,2MASS J11272881-3952572,220607.45,100,2016MNRAS.461..794P,B,Y*?,11 27 28.8046781424,-39 52 57.425386944,NaN,11.200,10.600,NaN,NaN,K3IV(e),4,0
3,4,HD 103823,201554.85,53,2011MNRAS.416.3108R,B,*,11 57 12.3975469056,-36 04 57.630203112,NaN,10.010,9.410,NaN,NaN,G2/3V,13,0
4,5,HD 102981,203345.92,82,2011MNRAS.416.3108R,B,*,11 51 13.1367947736,-43 55 59.005168680,NaN,6.583,6.593,NaN,NaN,A0V,35,0


In [16]:
df_ = pd.DataFrame()

In [20]:
ra= [ pd.to_numeric(ra.split(' ')) for ra in df['ICRS (J2000)RA'] if len(ra.split(' '))==3]

df_["ra_simbad"]= [ val[0] + val[1]/60 + val[2]/3600 for val in ra]

dec= [ pd.to_numeric(dec.split(' ')) for dec in df['ICRS (J2000)DEC'] if len(dec.split(' '))==3]

df_["dec_simbad"] = [ val[0] + val[1]/60 + val[2]/3600 if val[0]>0 else val[0] - val[1]/60 - val[2]/3600 for val in dec]

df_["Identifier"] = [ df["Identifier"][i] for i in range(len(df)) if  len(df['ICRS (J2000)DEC'][i].split(' '))==3]

In [21]:
df_.head()

,ra_simbad,dec_simbad,Identifier
0,11.242898,-44.306703,CD-43 6891
1,11.458161,-39.876512,HD 99627
2,11.458001,-39.882618,2MASS J11272881-3952572
3,11.953444,-36.082675,HD 103823
4,11.853649,-43.933057,HD 102981


In [19]:
df_.to_csv("sco_ob2_simbad.csv", index=False)

In [27]:
from astropy import units as u
from astroquery.xmatch import XMatch

In [33]:
table = XMatch.query(cat1=open('sco_ob2_simbad.csv'),
                     cat2='vizier:I/350/gaiaedr3',
                     max_distance=1.8*u.arcsec, colRA1='ra_simbad',
                     colDec1='dec_simbad')

In [34]:
table

angDist,ra_simbad,dec_simbad,Identifier,ra_epoch2000,dec_epoch2000,errHalfMaj,errHalfMin,errPosAng,source_id,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,astrometric_n_good_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,pseudocolour,pseudocolour_error,visibility_periods_used,ruwe,duplicated_source,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_mag,phot_rp_mean_flux,phot_rp_mean_mag,phot_bp_rp_excess_factor,bp_rp,dr2_radial_velocity,dr2_radial_velocity_error,dr2_rv_nb_transits,dr2_rv_template_teff,dr2_rv_template_logg,panstarrs1,sdssdr13,skymapper2,urat1,phot_g_mean_mag_error,phot_bp_mean_mag_error,phot_rp_mean_mag_error,phot_g_mean_mag_corrected,phot_g_mean_mag_error_corrected,phot_g_mean_flux_corrected,phot_bp_rp_excess_factor_corrected,ra_epoch2000_error,dec_epoch2000_error,ra_dec_epoch2000_corr
float64,float64,float64,str15,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.752395,12.414421862854,-72.60389868716,HD 107947,12.41439662079,-72.60368982498,0.0064917,0.0055969,31.94,4688982824084914688,12.41439957823,0.3019,-72.60369532371,0.2729,-0.1477,0.3217,-0.4592,1.253,0.199,0.361,-1.237,0.39,315,-1.4223,286.55,0.0,0.0,95,1.556,0.1042,26,0.942,0,214.257,0.805607,19.860027,139.773,5.66319,19.974987,103.2,19.713694,1.134,0.261293,--,--,0,--,--,--,--,--,--,0.004925,0.044079,0.082645,19.859676,0.004924,214.327,-0.031,5.8612,6.2541,0.1325
0.840701,13.525268722796667,-46.7352329795,HD 117484,13.52494038249,-46.73529538738,0.0007213,0.0004686,144.79,4981301596729062272,13.52479598777,0.0301,-46.73534723991,0.0347,2.6135,0.0564,46.3335,25.138,-22.267,0.035,-11.667,0.041,708,100.2785,15761.96,0.537,369.5,31,--,--,30,4.642,0,98090.2,36.9824,13.208303,50942.9,21.1408,13.570834,69330.0,12.645592,1.226,0.925242,--,--,0,--,--,--,--,503981259,--,0.002786,0.002826,0.003798,13.208303,0.002786,98090.2,0.016,0.5653,0.6483,-0.3866
1.024369,12.236527909716,-54.93395705264,TYC 8637-2220-1,12.23607181951,-54.9338461228,0.0057818,0.00458,2.45,4920844984233839104,12.23608701997,0.2291,-54.93387885352,0.2702,-0.6014,0.3292,-1.8268,7.622,1.965,0.285,-7.364,0.363,455,-0.3806,466.07,0.0,0.0,31,--,--,29,0.987,0,204.369,0.693383,19.911327,119.879,6.73439,20.141685,139.666,19.385166,1.27,0.756519,--,--,0,--,--,--,--,503331039,--,0.0046,0.061057,0.058879,19.911327,0.0046,204.369,0.074,4.5824,5.7798,0.0201
